In [ ]:
#|default_exp lgb_search

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# LightGBM search

In [ ]:
#|export
import copy
import os
from concurrent.futures import ThreadPoolExecutor
from typing import Any, Callable, Dict, List, Optional, Tuple

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.base import clone

from mlforecast import Forecast
from mlforecast.utils import backtest_splits

In [ ]:
#| exporti
class EarlyStopException(BaseException):
    ...

def _update_and_predict(fcst, n, h, valid):
    for _ in range(n):
        fcst.models_[0].update()
    preds = fcst.predict(h)
    preds = preds.merge(valid, on=['unique_id', 'ds'])
    preds['sq_err'] = (preds['Booster'] - preds['y']) ** 2
    rmse = preds.groupby('unique_id')['sq_err'].mean().pow(0.5).mean()
    return rmse

In [ ]:
#| export
class LGBMSearch:
    def __init__(
        self,
        freq: Optional[str] = None,  # pandas offset alias, e.g. D, W, M
        lags: List[int] = [],  # list of lags to use as features
        lag_transforms: Dict[int, List[Tuple]] = {},  # list of transformations to apply to each lag
        date_features: List[str] = [],  # list of names of pandas date attributes to use as features, e.g. dayofweek
        num_threads: int = 1,  # number of threads to use when computing the predictions of each window.
    ):
        self.num_threads = num_threads
        self.fcst = Forecast([], freq, lags, lag_transforms, date_features, 1)
        
    def _should_stop(self, hist, early_stopping_evals, early_stopping_pct):
        if len(hist) < early_stopping_evals + 1:
            return False
        # [8 5 4] (8 - 4) / 8 = 1 - 4 / 8 = 0.5
        improvement_pct = 1 - hist[-1][1] / hist[-(early_stopping_evals + 1)][1]
        return improvement_pct < early_stopping_pct

    def fit(
        self,
        data: pd.DataFrame,  # time series
        n_windows: int,  # number of windows to evaluate
        window_size: int,  # test size in each window
        params: Dict[str, Any] = {},  # lightgbm parameters
        id_col: str = 'index',  # column that identifies each serie, can also be the index.
        time_col: str = 'ds',  # column with the timestamps
        target_col: str = 'y',  # column with the series values
        static_features: Optional[List[str]] = None,  # column names of the features that don't change in time
        dropna: bool = True,  # drop rows with missing values created by lags
        keep_last_n: Optional[int] = None,  # keep only this many observations of each serie for computing the updates
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        weights: Optional[np.ndarray] = None,  # weight for each window
        eval_every: int = 10,  # number of iterations to train before evaluating the full window
        fit_on_all: bool = True,  # return model fitted on all data
        verbose_eval: bool = True,  # print evaluation metrics
        early_stopping_evals: int = 2,  # stop if the score doesn't improve in these many evaluations
        early_stopping_pct: float = 0.01,  # score must improve at least in this percentage to keep training
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn        
    ):
        if eval_every <= 0:
            raise ValueError(
                "eval_every should be > 0. If you don't want to evaluate the complete horizon use "
                "Forecast.backtest instead."
            )
        if weights is None:
            weights = np.full(n_windows, 1 / n_windows)        
        elif len(weights) != n_windows:
            raise ValueError('Must specify as many weights as the number of windows')
        else:
            weights = np.asarray(weights)
            
        orig = data.copy(deep=False)
        if id_col != 'index':
            data = data.set_index(id_col)
        data = data.rename(columns={time_col: 'ds', target_col: 'y'}, copy=False)
        data.index.name = 'unique_id'
        
        if np.issubdtype(data['ds'].dtype.type, np.integer):
            freq = 1
        else:
            freq = self.fcst.freq
        fcsts = []
        valids = []
        for _, train, valid in backtest_splits(data, n_windows, window_size, freq):
            cv_fcst = copy.deepcopy(self.fcst)
            prep = cv_fcst.preprocess(train, static_features=static_features)
            valids.append(valid)
            ds = lgb.Dataset(prep.drop(columns=['ds', 'y']), prep['y'])
            bst = lgb.Booster(params, ds)
            cv_fcst.models_ = [bst]
            fcsts.append(cv_fcst)

        hist = []
        n_iter = lgb.basic._choose_param_value('num_iterations', params, 100)['num_iterations']
        rmses = np.empty(n_windows)
        
        params['num_threads'] = os.cpu_count() // self.num_threads
        if self.num_threads == 1:
            try:
                for i in range(0, n_iter, eval_every):
                    for j, (fcst, valid) in enumerate(zip(fcsts, valids)):
                        rmses[j] = _update_and_predict(fcst, eval_every, window_size, valid)
                    rmse = rmses @ weights
                    rounds = eval_every + i
                    hist.append((rounds, rmse))
                    if verbose_eval:
                        print(f'[{rounds:,d}] RMSE: {rmse:,f}')                
                    if self._should_stop(hist, early_stopping_evals, early_stopping_pct):
                        raise EarlyStopException
            except EarlyStopException:
                print(f'Early stopping at round {rounds:,}')
        else:
            try:
                with ThreadPoolExecutor(self.num_threads) as executor:
                    for i in range(0, n_iter, eval_every):
                        futures = []
                        for fcst, valid in zip(fcsts, valids):
                            future = executor.submit(_update_and_predict, fcst, eval_every, window_size, valid)
                            futures.append(future)
                        rmses[:] = [f.result() for f in futures]
                        rmse = rmses @ weights
                        rounds = eval_every + i
                        hist.append((rounds, rmse))
                        if verbose_eval:
                            print(f'[{rounds:,d}] RMSE: {rmse:,f}')
                        if self._should_stop(hist, early_stopping_evals, early_stopping_pct):
                            raise EarlyStopException
            except EarlyStopException:
                print(f'Early stopping at round {rounds:,}.')
            
        if fit_on_all:
            self.fcst.models = [lgb.LGBMRegressor(**params)]
            self.fcst.fit(
                orig,
                id_col,
                time_col,
                target_col,
                static_features,
                dropna,
                keep_last_n,
            )
        return hist
    
    def predict(
        self,
        horizon: int,  # number of periods to predict in the future
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn
    ):
        return self.fcst.predict(horizon, dynamic_dfs, predict_fn, **predict_fn_kwargs)

In [ ]:
from mlforecast.utils import generate_daily_series
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean, seasonal_rolling_mean

In [ ]:
data = generate_daily_series(1_000, min_length=500, max_length=1_000)

In [ ]:
search = LGBMSearch(
    freq='D',
    lags=[i + 1 for i in range(4)],
    lag_transforms={
        7 : [expanding_mean] + [(rolling_mean, 7)],
        14: [expanding_mean] + [(rolling_mean, 14)],
    },
    num_threads=2,
)
%time search.fit(data, n_windows=2, window_size=14, params={'verbosity': -1})

[LightGBM] [Info] Start training from score 3.250137
[LightGBM] [Info] Start training from score 3.250088
[10] RMSE: 1.219636
[20] RMSE: 0.388444
[30] RMSE: 0.166221
[40] RMSE: 0.146804
[50] RMSE: 0.144275
[60] RMSE: 0.143951
[70] RMSE: 0.143915
Early stopping at round 70.
CPU times: user 23.8 s, sys: 1.23 s, total: 25.1 s
Wall time: 8.79 s


[(10, 1.21963550280172),
 (20, 0.38844393451941606),
 (30, 0.1662208308025986),
 (40, 0.14680427658840745),
 (50, 0.1442752213372901),
 (60, 0.143951352761174),
 (70, 0.14391519107862275)]

In [ ]:
search.predict(4)

,ds,LGBMRegressor
unique_id,,
id_000,2001-11-03,0.247302
id_000,2001-11-04,1.249704
id_000,2001-11-05,2.251363
id_000,2001-11-06,3.249986
id_001,2001-07-01,1.250885
...,...,...
id_998,2002-05-08,3.250413
id_999,2002-07-30,3.249795
id_999,2002-07-31,4.249041
